
Scikit-Learn
============

[Scikit-Learn](http://scikit-learn.org/) é a principal biblioteca para
aprendizado de máquina do Python. Sua API é extremamente simples e
bem-pensada. Diferente de muitas bibliotecas deste tipo, ela é um
exemplo de documentação e qualidade de código. Vamos ver o básico, mas
sinta-se encorajado a olhar a documentação e descobrir tudo que está
implementado.

Pré-processamento: atributos numérico
-------------------------------------

Conforme vimos em aula, é comum termos que realizar algumas
transformações nas bases de dados para que os dados estejam de acordo
com as premissas dos algoritmos. Vamos ver algumas transformaçoes usando
a API do sklearn. É importante ressaltar que todas as técnicas
implementadas na sklearn são organizadas utilizando Orientação à
Objetos. No caso das transformações, vamos ver inicialmente três
objetos: `sklearn.preprocessing.StandardScaler`,
`sklearn.preprocessing.MinMaxScaler`,
`sklearn.preprocessing.Normalizer`.

Independente da transformação, a API consiste de três métodos:

-   `fit`: realiza os ajustes para a função de transformação
    considerando a base de dados;
-   `transform`: recebe uma base de dados e retorna ela com a
    transformação aplicada;
-   `fit_transform`: faz os dois passos anteriores na mesma base de
    dados.

Considerando as três transformações comentadas acima, para
`StandardScaler` (que transforma um atributo para média zero e desvio
padrão um) é necessário computar o valor da média e desvio padrão,
`MinMaxScaler` (que transforma os valores de um atributo em uma faixa,
usualmente \[0,1\]) é necessário encontrar o menor e maior valor
observada, e por fim, no `Normalizer` (que normaliza objetos para terem
norma 1: $\|\mathbf{x}_i\|=1$) o passo `fit` não realiza nenhuma
operação.

In [ ]:
import pandas as pd
import sklearn.preprocessing as pp
#biblioteca com arrays N-dimensionais e computação científica em geral
import numpy as np

#criando um dataframe do pandas manualmente
X = pd.DataFrame({'A1': [1.,2.,3.,4.,5.], 'A2':[100.,200.,300.,41.,5.]})

Xscaled = pp.StandardScaler().fit_transform(X)
scaler = pp.StandardScaler()
scaler.fit(X)
#X.values corresponde ao array numpy que os dados do dataFrame estão usando
assert np.all(Xscaled == scaler.transform(X.values))
print("Médias de cada atributo original", scaler.mean_)

minmax = pp.MinMaxScaler()
Xmm = minmax.fit_transform(X)
print("X normalizado [0,1]:\n",Xmm)

norm = pp.Normalizer()
Xnorm = norm.transform(X)
print("X com norma 1 por linha:\n",Xnorm)


### Exercício: Verifique que Xscaled tem média zero e desvio padrão um para cada atributo

Pré-processamento: atributos categóricos
----------------------------------------

Como vimos em algumas aulas, nem todos os algoritmos sabem lidar bem com
atributos categóricos. Na verdade, essa é uma das áreas deficientes na
biblioteca sklearn. Os algoritmos implementados assumem valores
contínuos. Para poder usar os algoritmos com atributos categóricos,
vamos convertê-los em $V$ atributos binários, sendo $V$ o número de
diferentes valores que o atributo pode ter. Essa transformação é
denominada `OneHotEncoder`.

In [ ]:
enc = pp.OneHotEncoder()
X = [['estudante', 'computação', 'linux'], ['estudante', 'eiar', 'windows']]
enc.fit(X)
Xt = enc.transform([['estudante', 'computação', 'windows'],
                    ['estudante', 'eiar', 'linux']]).toarray()
print(Xt)

Pré-processamento: discretização
--------------------------------

Para discretizar um atributo numérico, podemos utilizar o
`KBinsDiscretizer`. Essa classe implementa os dois modos de
discretização visto em sala: largura fixa (`strategy=`\'uniform\') e
frequência fixa (`strategy=`\'quantile\').

In [ ]:
X2 = pd.DataFrame({'A1': [32,34,43,45,51,59,62,67,68,69,70,71,72]})
larg_fixa = pp.KBinsDiscretizer(n_bins = 8, strategy = 'uniform', encode = 'ordinal')
larg_fixa.fit(X2)
print(larg_fixa.transform(X2))
faixas = [ f"[{x:.0f},{y:.0f})" for x,y in zip(larg_fixa.bin_edges_[0], larg_fixa.bin_edges_[0][1:])]
print("Intervalos:" + ", ".join(faixas))


In [ ]:
X2 = pd.DataFrame({'A1': [32,34,43,45,51,59,62,67,68,69,70,71,72]})
freq_fixa = pp.KBinsDiscretizer(n_bins = 5, strategy = 'quantile', encode = 'ordinal')
freq_fixa.fit(X2)
print(freq_fixa.transform(X2))
faixas = [ f"[{x:.0f},{y:.0f})" for x,y in zip(freq_fixa.bin_edges_[0], freq_fixa.bin_edges_[0][1:])]
print("Intervalos:" + ", ".join(faixas))

Scikit-Learn - Parte 2
======================

Vimos anteriormente como utilizar a biblioteca Scikit-Learn para realizar o pré-processamento dos dados.
Veremos agora o básico sobre como rodar algoritmos de classificação e regressão nessa biblioteca.

Principais métodos
------------------

Os algoritmos de aprendizado de máquina implementados na Scikit-Learn utilizam uma API bastante intuitiva, você trabalhará frequentemente com três métodos:
- `fit`: treina um modelo, para algoritmos de aprendizado supervisionado você deve passar dois parâmetros `X` (matriz de dados que pode ser um `pandas.DataFrame`) e `y` (vetor com rótulos dos dados, pode ser uma coluna de um `pandas.DataFrame`, também conhecido como `pandas.Series`);
- `predict`: realiza a predição para novos dados, recebe como primeiro parâmetro uma matriz de objetos de teste;
- `predict_proba`: similar ao `predict`, mas ao invés de retornar apenas os rótulos de classe preditas, retorna a probabilidade _posteriori_ de cada classe, ou seja, $p(c|x)$.

Um breve exemplo
----------------

Vamos ver como treinar o modelo Naïve Bayes considerando todos os atributos distribuídos de acordo com uma distribuição Gaussiana.
Este algoritmo está implementado como `GaussianNB` no módulo `sklearn.naive_bayes`.
Vamos utilizar a base de dados Iris que tem 150 objetos, 4 atributos e 3 classes.


In [ ]:
import pandas as pd
from sklearn.naive_bayes import GaussianNB

dados = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data", header = None)
print(dados.head())
#sklearn trabalha com X e y separados
X, y = dados.drop(4, axis = 1), dados[4]
clf = GaussianNB()
clf.fit(X,y)


Podemos examinar o modelo por meio dos atributos do objeto `clf`. Por exemplo, podemos ver a média estimada para cada atributo em `clf.theta_`. Como temos 3 classes e 4 atributos, este atributo corresponde a uma matriz 3x4.

In [ ]:
print(clf.theta_)

Você pode ver todas as informações armazenadas no modelo na [documentação dessa classe](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html).
Podemos examinar o erro obtido por esse modelo considerando os dados de treinamento por meio do `predict`.

In [ ]:
predicoes = clf.predict(X)
qtd_erros = predicoes != y
taxa_erro = qtd_erros.sum() / X.shape[0]
print(f"Erro de treinamento: {taxa_erro:.3f}")

Esse erro não é muito informativo, seria melhor vermos quanto ele erra em dados que não foram usados no treinamento. Para ajudar nisso, podemos utilizar [StratifiedShuffleSplit](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedShuffleSplit.html#sklearn.model_selection.StratifiedShuffleSplit).
Com ele, temos um conjunto de divisões aleatórias da base de dados que mantém a proporção de objetos de cada classe em cada divisão.


In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
divisoes = StratifiedShuffleSplit(n_splits=2, test_size=0.5, random_state=0)
for idx_treino, idx_teste in divisoes.split(X, y):
    print("Objetos no treino:", idx_treino)
    print("Objetos no teste:", idx_teste)
    Xtreino, ytreino = X.iloc[idx_treino], y.iloc[idx_treino]
    Xteste, yteste = X.iloc[idx_teste], y.iloc[idx_teste]
    erros = clf.fit(Xtreino, ytreino).predict(Xteste) != yteste
    print(f"Erro de teste: {erros.sum()/len(idx_teste):.3f}")


É possível notar um pequeno aumento na taxa de erro. Isso é esperado. A taxa de erro no treinamento tende a ser uma estimativa otimista, afinal, aqueles dados foram usados para treinar o modelo.

Note que o método `fit` retorna o próprio objeto, portanto, podemos utilizar os métodos por meio de uma [interface fluente](https://en.wikipedia.org/wiki/Fluent_interface).
Um conjunto considerável de algoritmos está implementado [na biblioteca](https://scikit-learn.org/stable/supervised_learning.html). O fato deles usarem a mesma API facilita bastante seu uso.


Visualizando fronteiras de decisão
==================================

Como vimos nas aulas, quando temos apenas dois atributos é interessante visualizar a fronteira de decisão entre as classes.
Uma forma simples, que funciona para qualquer modelo independente da complexidade de sua fronteira, é realizar a predição para cara ponto em um _grid_ pré-definido.

Para ilustrar isso, transformaremos os atributos originais da base Iris (comprimento da sépala, largura da sépala, comprimento da pétala e largura da pétala) em atributos de área e consideramos pontos entre os limites de valores presentes na base de dados.

In [ ]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
iris,cls = load_iris(return_X_y=True)
novaIris = pd.DataFrame({'area_sepala': iris[:,0] * iris[:,1],
                         'area_petala': iris[:,2] * iris[:,3]})

print(novaIris.head())

xMin, xMax = novaIris['area_sepala'].min() - 1, novaIris['area_sepala'].max() + 1
yMin, yMax = novaIris['area_petala'].min() - 1, novaIris['area_petala'].max() + 1

xx, yy = np.meshgrid(np.arange(xMin - 0.1, xMax + 0.1, 0.1),
                     np.arange(yMin - 0.1, yMax + 0.1, 0.1))

modelo = KNeighborsClassifier(n_neighbors = 7)
modelo.fit(novaIris, cls)

ax = plt.gca()
Z = modelo.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)
ax.contourf(xx, yy, Z, alpha=0.4)
ax.scatter(novaIris['area_sepala'], novaIris['area_petala'], c=cls, s=20, edgecolor='k')

Validação cruzada
==================================

O Scikit-Learn possui diversos utilitários para auxiliar na avaliação de modelos por meio de validação cruzada. Sugiro que você leia [esta página que os descreve](https://scikit-learn.org/stable/modules/cross_validation.html).

Veremos dois usos comuns:
1. Obter o $E_{CV}$, ou seja, erro médio de validação cruzada;
2. Como ter o laço de controle da validação cruzada, tendo assim acesso à quais objetos estão em cada pasta.





## Obtendo $E_{CV}$

Para isso podemos usar a função `cross_val_score`.
O método recebe quatro parâmetros:
- Um objeto com o classificador que deverá ser usado (ele será re-treinado a cada iteração da validação cruzada);
- A matriz de dados;
- O vetor com as saídas esperadas;
- O número de pastas que devem ser utilizadas (parâmetro `cv`).

[Como pode ser visto aqui](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html#sklearn.model_selection.cross_val_score), essa função tem diversos outros parâmetros que controlem inclusive o paralelismo de execução, mas estes sãos os básicos para o que queremos.
A função retorna o erro, por padrão usando a medida de avaliação *acurácia*, obtido em cada iteração.



In [ ]:
from sklearn.model_selection import cross_val_score
clf = KNeighborsClassifier(n_neighbors=1)
scores = cross_val_score(clf, novaIris, cls, cv=5)
print(scores)

## Laço de validação cruzada


Por vezes você vai precisar ter controle sobre os dados dentro do laço principal de validação cruzada, por exemplo, quando quer testar vários modelos e analisar os erros obtidos em cada pasta.

A classe `StratifiedKFold` pode auxiliar nisso. Ela realiza a divisão dos objetos considerando a proporção dos objetos de cada classe. Em outras palavras, a mesma proporção de objetos em cada classe na base original é refletida em cada uma das pastas.
Seu uso é bem simples, funciona similar ao `StratifiedShuffleSplit` que vimos antes.



In [ ]:
from sklearn.model_selection import StratifiedKFold

kf = StratifiedKFold(n_splits = 10)
for idx_treino, idx_teste in kf.split(novaIris, cls):
    Xtreino, ytreino = novaIris.iloc[idx_treino], cls[idx_treino]
    Xteste, yteste = novaIris.iloc[idx_teste], cls[idx_teste]
    erros = clf.fit(Xtreino, ytreino).predict(Xteste) != yteste
    print(f"Erro de teste: {erros.sum()/len(idx_teste):.3f}")


# Seleção de modelos

Vimos em aula que um procedimento muito utilizado para a seleção de modelos é a validação cruzada.
Basicamente, tínhamos que realizar validação cruzada para cada algoritmo (com sua configuração de hiperparâmetros) e escolher aquele que apresentava o menor $E_{CV}$.

Implementar isso para pode ser um pouco tedioso, felizmente temos o `GridSearchCV` que nos permite fazer exatamente isso!

A ideia consiste em termos um *grid* de parâmetros contendo todos os parâmetros que queremos variar para um algoritmo. O `GridSearchcV` vai simplesmente rodar todos (busca exaustiva) e selecionar o melhor.
Após selecionar o melhor, ele automaticamente (configurável via parâmetro), re-treina o modelo com todos os dados, **como deve ser feito**.

[Vale a pena conhecer um pouco mais sobre essa funcionalidade no scikit-learn](https://scikit-learn.org/stable/modules/grid_search.html).

[Caso queira usar o `GridSearchCV` para diferentes classificadores, vale a pena você conhecer sobre o `Pipeline`](https://scikit-learn.org/stable/modules/compose.html#pipeline).

In [ ]:
from sklearn.model_selection import GridSearchCV
from pprint import PrettyPrinter
pp = PrettyPrinter()

prm_grid = [ #cada elemento dessa lista é um dicionário com os parâmetros que devem ser buscados em exaustão e seus limites
    {'n_neighbors': [1, 3, 5, 7]}
]
knn = KNeighborsClassifier()
#validação cruzada de 5 pastas
cv = GridSearchCV(knn, prm_grid, cv = 5)
cv.fit(novaIris, cls)
print("Resultados encontrados na validação cruzada")
pp.pprint(cv.cv_results_)

print("Melhor parâmetro identificado")
print(cv.best_params_)
print("Classificador re-treinado")
print(cv.best_estimator_)
